In [0]:
!kaggle competitions download -c jigsaw-toxic-comment-classification-challenge

 95% 25.0M/26.3M [00:00<00:00, 40.4MB/s]
100% 26.3M/26.3M [00:00<00:00, 41.5MB/s]
 81% 19.0M/23.4M [00:00<00:00, 23.3MB/s]
100% 23.4M/23.4M [00:00<00:00, 30.1MB/s]
  0% 0.00/1.46M [00:00<?, ?B/s]
100% 1.46M/1.46M [00:00<00:00, 208MB/s]
  0% 0.00/1.39M [00:00<?, ?B/s]
100% 1.39M/1.39M [00:00<00:00, 194MB/s]


In [0]:
!unzip /content/train.csv.zip
!unzip /content/test.csv.zip

Archive:  /content/train.csv.zip
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv               
Archive:  /content/test.csv.zip
  inflating: test.csv                


In [0]:
import pandas as pd
import numpy as np

In [0]:
embed_size = 50 # how big is each word vector
max_features = 20000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 100 # max number of words in a comment to use

In [0]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
list_sentences_train = train["comment_text"].fillna("_na_").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("_na_").values

In [0]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

In [0]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_t = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)

In [0]:
f=open("glove.6B.50d.txt","r")

In [0]:
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.strip().split()) for o in open("/content/drive/My Drive/glove.6B.50d.txt","r"))

In [0]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
emb_mean,emb_std

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if self.run_code(code, result):


(0.020940498, 0.6441043)

In [0]:
import pickle
f=open("word2idxcomments2.pkl","wb")
pickle.dump(word_index,f)
f.close

<function BufferedWriter.close>

In [0]:
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [0]:
f=open("embeddingweightsforcomment.txt","w")
f.write(str(embedding_matrix))
f.close

<function TextIOWrapper.close>

In [0]:
embedding_matrix[2]

array([ 0.68046999, -0.039263  ,  0.30186   , -0.17792   ,  0.42962   ,
        0.032246  , -0.41376001,  0.13228001, -0.29846999, -0.085253  ,
        0.17117999,  0.22419   , -0.10046   , -0.43652999,  0.33418   ,
        0.67846   ,  0.057204  , -0.34448001, -0.42785001, -0.43274999,
        0.55962998,  0.10032   ,  0.18677001, -0.26853999,  0.037334  ,
       -2.09319997,  0.22171   , -0.39868   ,  0.20912001, -0.55725002,
        3.88260007,  0.47466001, -0.95657998, -0.37788001,  0.20869   ,
       -0.32752001,  0.12751   ,  0.088359  ,  0.16350999, -0.21634001,
       -0.094375  ,  0.018324  ,  0.21048   , -0.03088   , -0.19722   ,
        0.082279  , -0.09434   , -0.073297  , -0.064699  , -0.26043999])

In [0]:
embeddinweights=embedding_matrix

In [0]:
from keras.models import Sequential
from keras.layers import *


In [0]:
model=Sequential()
model.add(Embedding(input_dim=max_features,output_dim=embed_size,input_length=X_t.shape[1]))
model.add(Bidirectional(LSTM(50,dropout=0.1,recurrent_dropout=0.1,return_sequences=True)))
model.add(GlobalMaxPool1D())
model.add(Dense(50,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(6,activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 50)           1000000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 100)          40400     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                5050      
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 306       
Total params: 1,045,756
Trainable params: 1,045,756
Non-trainable params: 0
____________________________________________

In [0]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
from keras.callbacks import EarlyStopping,ModelCheckpoint
early=EarlyStopping(monitor='val_loss',patience=2,mode='auto')
modelcheck=ModelCheckpoint("bestnewmodel.h5",save_best_only=True,monitor='val_accuracy',mode='auto')

In [0]:
hist=model.fit(X_t,y,epochs=5,validation_split=0.2,callbacks=[early,modelcheck])

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 127656 samples, validate on 31915 samples
Epoch 1/5
127656/127656 [==============================] - 1435s 11ms/step - loss: 0.0663 - accuracy: 0.9784 - val_loss: 0.0498 - val_accuracy: 0.9821
Epoch 2/5
127656/127656 [==============================] - 1429s 11ms/step - loss: 0.0467 - accuracy: 0.9828 - val_loss: 0.0498 - val_accuracy: 0.9821
Epoch 3/5
127656/127656 [==============================] - 1425s 11ms/step - loss: 0.0427 - accuracy: 0.9839 - val_loss: 0.0493 - val_accuracy: 0.9821
Epoch 4/5
127656/127656 [==============================] - 1417s 11ms/step - loss: 0.0386 - accuracy: 0.9852 - val_loss: 0.0515 - val_accuracy: 0.9823
Epoch 5/5
127656/127656 [==============================] - 1424s 11ms/step - loss: 0.0349 - accuracy: 0.9863 - val_loss: 0.0563 - val_accuracy: 0.9822


In [0]:
model.save_weights("bestestmodel.h5")

In [0]:
model.save("ToxicCommentsmodel.h5")